In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import umap
import hdbscan
from tslearn.metrics import dtw
from pynndescent import NNDescent # pynndescent 라이브러리 임포트

# --- 1. 설정 및 데이터 로드 ---
# -----------------------------
DATASET_PATH = './dataset/preprocessed_training_dataset.npy'

# 계산 시간이 매우 길 수 있으므로, 먼저 작은 샘플로 테스트하는 것을 강력히 권장합니다.
SAMPLE_SIZE = 1000

print("--- Step 1: Loading and Sampling Data ---")
X_train = np.load(DATASET_PATH)
sample_indices = np.random.choice(X_train.shape[0], SAMPLE_SIZE, replace=False)
X_sample = X_train[sample_indices]
X_sample_2d = X_sample.reshape(SAMPLE_SIZE, -1)
print(f"✅ Data prepared. Shape: {X_sample_2d.shape}")

--- Step 1: Loading and Sampling Data ---
✅ Data prepared. Shape: (1000, 1440)


In [6]:
# Numba JIT 컴파일러를 직접 사용하기 위해 임포트
from numba import njit, float32
from pynndescent import NNDescent

@njit(fastmath=True)
def numba_dtw_independent(s1, s2):
    """
    Numba로 컴파일 가능한 독립적인 DTW 함수.
    tslearn에 의존하지 않아 안정적입니다.
    """
    n, m = len(s1), len(s2)
    # 비용 행렬 초기화 (첫 행과 열은 무한대로)
    cost_matrix = np.full((n + 1, m + 1), np.inf)
    cost_matrix[0, 0] = 0.

    # 동적 프로그래밍으로 비용 행렬 채우기
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = (s1[i - 1] - s2[j - 1]) ** 2
            cost_matrix[i, j] = cost + min(cost_matrix[i - 1, j],        # Insertion
                                          cost_matrix[i, j - 1],        # Deletion
                                          cost_matrix[i - 1, j - 1])    # Match
    
    # 최종 DTW 거리 반환
    return np.sqrt(cost_matrix[n, m])

In [ ]:
from joblib import dump

# --- 2. (★★★ 핵심 수정 사항 ★★★) pynndescent로 사전 인덱싱 ---
# -------------------------------------------------------------
print("\n--- Step 2: Pre-computing approximate nearest neighbors with pynndescent ---")
print("This pre-computation makes the main UMAP step much faster.")

# pynndescent를 사용하여 DTW 기반의 최근접 이웃 인덱스를 미리 생성
# 이 과정이 UMAP에 직접 DTW를 넣는 것보다 훨씬 빠릅니다.
index = NNDescent(
    X_sample_2d,
    metric=numba_dtw_independent,
    n_neighbors=20, # UMAP의 n_neighbors와 동일하게 설정
    random_state=42,
    n_jobs=-1
)
index.prepare()

output_path = './output/pynndescent_index.joblib'
dump(index, output_path)
print(f"✅ Index saved to {output_path}")


--- Step 2: Pre-computing approximate nearest neighbors with pynndescent ---
This pre-computation makes the main UMAP step much faster.
